## LCG - Linear ... Generator

Um dos primeiros e mais conhecidos métodos de geração de números pseudos aleatórios.
$$X_{n+1} = (aX_n + c)\ mod\ m$$

Onde:

- $X_0: Seed$
- $c : Incremento$
- $a: Multiplicador$
- $m: Modulando$

Coisas interessantes:
$$x\ mod\ 1=0, ∀x∈\R\\ x\ mod\ a = r\ |\  0\le r \lt a\\ \text{ se r fosse maior que } a\text{, daria para fazer mais uma divisão.}  $$

In [2]:
using GLMakie

In [3]:
function lcg(seed, steps, multiplier, increment, modulando)
    random_num = seed
    for _ in 1:steps
        random_num = (multiplier*random_num + increment) % modulando
    end
    random_num
end

lcg (generic function with 1 method)

In [9]:
interval = 1:10000
hardcoded_generated_nums = [lcg(4521, i, 521, 2124, 94821)/94821 for i ∈ interval]

10000-element Vector{Float64}:
 0.8633214161419939
 0.8128579112221976
 0.5213718480083526
 0.6571329135950897
 0.3886480842851267
 0.5080520137944127
 0.7174992881323757
 0.8395292182111558
 0.41712278925554463
 0.34337330338216215
 ⋮
 0.7499604517986522
 0.7517954883411903
 0.7078495270035119
 0.812003670073085
 0.0763122093207201
 0.7810611573385643
 0.9552630746353656
 0.7144619862688645
 0.25709494732179583

In [43]:
import Makie: ComputePipeline
# if !@isdefined(GLMakie)
#     using GLMakie
# end

# Crio uma pipeline de computação reativa
cmpt_graph = ComputePipeline.ComputeGraph()

fig = Figure(size = (1720, 1720))

# Crio um grid de sliders
sliders = [
    (label="Interval Size", range= 1:10000, startvalue=100, snap = false),
    (label=L"X_0", range = -10:100000, startvalue = 1, snap = false),
    (label="Multiplier", range = 1:100000, startvalue = 2, snap = false),
    (label="Increment", range = -1000:10000, startvalue = 0, snap = false),
    (label="Modulando", range = 1:100000, startvalue = 128, snap = false),
]

slider_grid = SliderGrid(fig, sliders...)

observable_names = :(interval_size_obs, seed_obs, multiplier_obs, increment_obs, modulando_obs) 
# Pego o primeiro observable do grid de slider e associo a uma variável.
observables = [slider.value for slider in slider_grid.sliders]

for (name, value) in zip(observable_names.args, observables)
    # Adiciono esse observable a pipeline de de computação
    ComputePipeline.add_input!(cmpt_graph, name, value)
end

# Toda vez que a variável atualizar, a função abaixo é executada e seu resultado é armazenado no observable generated.
ComputePipeline.map!(
    (in1, in2, in3, in4, in5) -> begin
        return (
            [lcg(in2, i, in3, in4, in5) / in5 for i in 1:in1],
        )
    end,
    cmpt_graph,
    observable_names.args,
    [:generated_obs]
)

for name in observable_names.args
    println(name)
    eval(:( $(name) = cmpt_graph[$(QuoteNode(name))] ))
end

generated = cmpt_graph[:generated_obs]
# interval = cmpt_graph[:interval_size_obs]

formula_obs = Makie.lift(interval_size_obs, seed_obs, multiplier_obs, increment_obs, modulando_obs) do interval_size_obs, seed_obs, multiplier_obs, increment_obs, modulando_obs
    L"X_n = \frac{(%$multiplier_obs*X_{n-1}+%$increment_obs)\ mod\ %$modulando_obs}{%$modulando_obs}"
end

formula_label = Label(fig[1, 2],
    formula_obs,
    tellwidth = true,
    halign = :center,
    valign = :center,
    fontsize = 28
)

axis = Axis(fig, xminorgridvisible=true, yminorgridvisible=true,
    xminorticksvisible=true, xminorticks=IntervalsBetween(10))

axis.limits = ((1,150),(-0.5, 1.5))

fig[1, 1:2] = axis
fig[2, 2] = slider_grid
fig[2, 1] = formula_label

rowsize!(fig.layout, 1, Auto())
colsize!(fig.layout, 1, Relative(1/2))

scatterlines!(
    axis,
    lift(interval_size_obs, generated) do n, y
        [Point2(i, y[i]) for i in 1:n]
    end,
)
DataInspector(fig)

interval_size_obs
seed_obs
multiplier_obs
increment_obs
modulando_obs


DataInspector(Scene(3 children, 1 plots), Attributes(), Dict{Tuple{Scene, Type}, Plot}(), Plot[], Tooltip{Tuple{Point{2, Float32}}}, Tooltip{Tuple{Point{2, Float32}}}, Any[ObserverFunction defined at /home/gfgzk/.julia/packages/Observables/YdEbO/src/Observables.jl:419 operating on Observable((0.0, 0.0)), ObserverFunction defined at /home/gfgzk/.julia/packages/Observables/YdEbO/src/Observables.jl:419 operating on Observable((0.0, 0.0))], Channel{Nothing}(9223372036854775807))

In [44]:
display(fig)

GLMakie.Screen(...)

Vou multiplicar os valores obtidos por 10, truncar os resultados e calcular a probabilidade de cada um. Veremos que se aproxima de uma distribuição uniforme.

In [10]:
hardcoded_generated_nums_integer = trunc.(Int, hardcoded_generated_nums.*(10))
prob_dist = Array{Union{Missing, Float64}}(missing, 10)
for n ∈ 0:9
    prob_dist[n+1] = count(==(n), hardcoded_generated_nums_integer)/length(hardcoded_generated_nums_integer)
end


In [12]:
fig_2 = Figure(size = (400, 250))

ax = Axis(
    fig_2[1, 1],
    xlabel = "Valor",
    ylabel = "Probabilidade",
    aspect = nothing
)

barplot!(ax, 0:9, prob_dist, gap = 0.1)
ax.xticks = 0:9

display(fig_2)
prob_dist


10-element Vector{Union{Missing, Float64}}:
 0.1022
 0.0966
 0.0988
 0.0999
 0.1059
 0.1007
 0.1016
 0.0966
 0.1005
 0.0972

Vamos usar esse gerador de número pseudo aleatório para retirar bolas de uma urna.
A urna é representada pelo array [v, v, v, a, a, a, a, a]. Usamos nosso LCG para escolher um índice aleatório desse vetor.

In [ ]:
bolas = [repeat('v', 3)..., repeat('a', 5)...]
indice_sorteado = hardcoded_generated_nums_integer[1]
bolas[indice_sorteado]

'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)

Dado um evento com probabilidade arbitrária p = 0.7

In [35]:
aconteceu = 0.7 > hardcoded_generated_nums[9]


true